<a href="https://colab.research.google.com/github/alohia/pytorch_playground/blob/master/03_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U transformers sklearn torchtext

Requirement already up-to-date: transformers in /usr/local/lib/python3.6/dist-packages (2.2.1)
Requirement already up-to-date: sklearn in /usr/local/lib/python3.6/dist-packages (0.0)
Requirement already up-to-date: torchtext in /usr/local/lib/python3.6/dist-packages (0.4.0)


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import transformers as tfms
import torch
from tqdm import tqdm_notebook as tn
import pickle
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import warnings
warnings.filterwarnings("ignore")

In [0]:
df = pd.read_csv('https://www.dropbox.com/s/1kz3wkni6tyns8a/yelp_train.csv?dl=1')

In [4]:
df.shape

(50000, 2)

In [0]:
sample = df[:2000]

In [6]:
sample.head()

,y,text
0,2,With such a highly esteemed name for a restaur...
1,4,I came here on American Thanksgiving. Had a sm...
2,3,I am going to agree with the review of Juddi L...
3,3,"Meh. These are nice but not spectacular, howe..."
4,2,One thing my wife and I try to avoid on date n...


In [7]:
sample['y'].value_counts()

5    412
2    408
4    397
1    395
3    388
Name: y, dtype: int64

In [0]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (tfms.DistilBertModel, tfms.DistilBertTokenizer, 'distilbert-base-uncased')

# For BERT:
# model_class, tokenizer_class, pretrained_weights = (tfms.BertModel, tfms.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [0]:
tokenized = sample['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=512)))

In [0]:
max_len = max(tokenized.apply(len))

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [11]:
padded.shape

(2000, 512)

In [12]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(2000, 512)

In [0]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

In [0]:
padded = torch.tensor(padded)
attention_mask = torch.tensor(attention_mask)

In [0]:
train_ds = TensorDataset(padded, attention_mask, torch.tensor(sample['y']))
train_dl = DataLoader(train_ds, batch_size=256, shuffle=True)

In [16]:
use_cuda = torch.cuda.is_available()
device = torch.device('cuda:0' if use_cuda else 'cpu')
model = model.to(device)
ftrs = []
labs = []
for i, m, y in tn(train_dl):
    input_ids = i.to(device)
    attention_mask = m.to(device)
    with torch.no_grad():
        last_hidden_states = model(input_ids, attention_mask=attention_mask)[0]
        ftrs.append(last_hidden_states[:,0,:].cpu().numpy())
        labs.append(y.numpy())

In [0]:
features = np.concatenate(ftrs)
np.save('bert_features.npy', features)

In [0]:
labels = np.concatenate(labs)
np.save('bert_labels.npy', labels)

In [0]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, random_state=42, stratify=labels)

In [20]:
parameters = {'C': np.linspace(0.001, 100, 20)}
grid_search = GridSearchCV(LogisticRegression(), parameters, n_jobs=-1)
grid_search.fit(train_features, train_labels)

print('best parameters: ', grid_search.best_params_)

best parameters:  {'C': 5.264105263157894}


In [21]:
lr_clf = LogisticRegression(C=grid_search.best_params_['C'])
lr_clf.fit(train_features, train_labels)

LogisticRegression(C=5.264105263157894, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [22]:
lr_clf.score(train_features, train_labels)

0.888

In [23]:
lr_clf.score(test_features, test_labels)

0.494

In [0]:
from sklearn.svm import SVC

In [25]:
parameters = {'C': np.linspace(0.001, 100, 20)}
grid_search = GridSearchCV(SVC(), parameters, n_jobs=-1)
grid_search.fit(train_features, train_labels)

print('best parameters: ', grid_search.best_params_)

best parameters:  {'C': 21.053421052631577}


In [26]:
svc_clf = SVC(C=grid_search.best_params_['C'])
svc_clf.fit(train_features, train_labels)

SVC(C=21.053421052631577, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [27]:
svc_clf.score(train_features, train_labels)

0.6413333333333333

In [28]:
svc_clf.score(test_features, test_labels)

0.516

## Use Glove embeddings

In [0]:
from torchtext.vocab import GloVe

In [18]:
glove = GloVe(name = '6B', dim = 300)

.vector_cache/glove.6B.zip: 862MB [06:30, 2.21MB/s]                           
100%|█████████▉| 399432/400000 [00:47<00:00, 8134.20it/s]

In [0]:
np.save('embed.npy', glove.vectors.numpy())

In [0]:
pickle.dump(glove.stoi, open('stoi.pkl', 'wb'))

In [33]:
embeddings = np.load('embed.npy')
word_to_indx = pickle.load(open('stoi.pkl', 'rb'))

100%|█████████▉| 399883/400000 [01:00<00:00, 8097.06it/s]

In [0]:
vocab_size, emb_sz = embeddings.shape

In [35]:
vocab_size, emb_sz

(400000, 300)

In [0]:
svm_df = sample.copy()
svm_df['text'] = svm_df.text.str.lower()
svm_df['embs'] = svm_df.text.apply(lambda x:np.array([embeddings[word_to_indx.get(word, 0)] for word in x.split()]).mean(0))

In [0]:
features = np.stack(svm_df['embs'].values)
np.save('svm_features.npy', features)

In [0]:
labels = np.stack(svm_df['y'].values)
np.save('svm_labels.npy', labels)

In [39]:
features.shape, labels.shape

((2000, 300), (2000,))

In [0]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, random_state=42, stratify=labels)

In [0]:
from sklearn.svm import SVC

In [42]:
parameters = {'C': np.linspace(0.001, 100, 20)}
grid_search = GridSearchCV(SVC(), parameters, n_jobs=-1)
grid_search.fit(train_features, train_labels)

print('best parameters: ', grid_search.best_params_)

best parameters:  {'C': 100.0}


In [43]:
svc_clf = SVC(C=grid_search.best_params_['C'])
svc_clf.fit(train_features, train_labels)

SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [44]:
svc_clf.score(train_features, train_labels)

0.5293333333333333

In [45]:
svc_clf.score(test_features, test_labels)

0.424

## Using custom head on frozen BERT

In [0]:
clf_model = tfms.DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=5)

In [0]:
true_labs = torch.tensor(sample['y'] - 1)

In [0]:
model

In [0]:
clf_model

In [16]:
padded.shape, attention_mask.shape

(torch.Size([2000, 512]), torch.Size([2000, 512]))

In [56]:
padded, attention = pickle.load(open('bert_pad_attention', 'rb'))

(2000, 512)

In [0]:
train_padded, train_attention, train_y = padded[:1500], attention_mask[:1500], true_labs[:1500]
test_padded, test_attention, test_y = padded[1500:], attention_mask[1500:], true_labs[1500:]

In [0]:
train_ds = TensorDataset(train_padded, train_attention, train_y)
train_dl = DataLoader(train_ds, batch_size=128, shuffle=True)

valid_ds = TensorDataset(test_padded, test_attention, test_y)
valid_dl = DataLoader(valid_ds, batch_size=128)

In [0]:
class myBertClassifier(nn.Module):
  def __init__(self, bert_model, n_hid):
    super().__init__()
    self.n_hid = n_hid
    self.bert = bert_model
    self.fc1 = nn.Linear(768, self.n_hid)
    self.fc2 = nn.Linear(self.n_hid, 5)
    for p in self.bert.parameters():
      p.requires_grad = False

  def forward(self, xb, am):
    bert_out = self.bert(xb, am)[0]
    bert_last_out = bert_out[:,0,:]
    pre_clas = self.fc1(bert_last_out)
    out = self.fc2(F.relu(pre_clas))
    return out

In [0]:
mbc = myBertClassifier(model, 100)

In [0]:
use_cuda = torch.cuda.is_available()
device = torch.device('cuda:0' if use_cuda else 'cpu')
mbc = mbc.to(device)

In [0]:
epochs = 3
loss_func = F.cross_entropy
def accuracy(out, yb):
    preds = torch.argmax(out, dim=1)
    return (preds == yb).float().mean()

In [76]:
opt = optim.Adam(mbc.parameters(), lr=0.01)

for epoch in tn(range(epochs)):
    mbc.train()
    for xb, ab, yb in tn(train_dl):
        xb = xb.to(device)
        ab = ab.to(device)
        yb = yb.to(device)
        pred = mbc(xb, ab)
        loss = loss_func(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()

    mbc.eval()
    losses = []
    accs = []
    with torch.no_grad():
      for xb, ab, yb in valid_dl:
        xb = xb.to(device)
        ab = ab.to(device)
        yb = yb.to(device)
        pred = mbc(xb, ab)
        batch_loss = loss_func(pred, yb)
        batch_acc = accuracy(pred, yb)
        losses.append(batch_loss.item())
        accs.append(batch_acc.item())

    print(epoch, 'loss', np.mean(losses), 'acc', np.mean(accs))

0 loss 1.3724533021450043 acc 0.42632004618644714
1 loss 1.1687602698802948 acc 0.4724542051553726
2 loss 1.1254068613052368 acc 0.4628232717514038


In [0]:
pickle.dump((padded, attention_mask), open('bert_pad_attention', 'wb'))

In [0]:
import time

In [90]:
for i in tn(range(500)):
  print('a', end=' ')
  if i%50 == 0:
    print()
  time.sleep(1)

a 
a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a 
a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a 
a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a 
a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a 
a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a 
a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a 
a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a 
a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a 
a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a 
a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a 

In [0]:
xb, ab, yb = next(iter(train_dl))

In [59]:
xb.shape

torch.Size([128, 512])

In [60]:
yb.shape

torch.Size([128])

In [61]:
ab.shape

torch.Size([128, 512])

In [0]:
xb = xb.to(device)
ab = ab.to(device)

In [0]:
g = mbc(xb, ab)

In [64]:
g.shape

torch.Size([128, 5])